```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Interpolation errors

Goal of this notebook: To $N(0,I)$Â random numbers on a coarse grid, apply the SOAR function, and then interpolate the result down to the fine computational grid.


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, IPythonMagic

from SWESimulators import OceanStateNoise, GPUDrifterCollection


In [ ]:
%cuda_context_handler gpu_ctx
gpu_stream = cuda.Stream()

In [ ]:
def imshow(im, interpolation="None", title=None, figsize=(4,4)):
    fig = plt.figure(figsize=figsize)
    plt.imshow(im, interpolation=interpolation, origin='lower')
    plt.colorbar()
    if title is not None:
        plt.title(title)
        
def imshow3(eta, hu, hv, interpolation="None", title=None):
    fig, axs = plt.subplots(1,3, figsize=(12,4))
    
    eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower')
    axs[0].set_title("eta")
    plt.colorbar(eta_im, ax=axs[0])
    
    hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower')
    axs[1].set_title("hu")
    plt.colorbar(hu_im, ax=axs[1])

    
    hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower')
    axs[2].set_title("hv")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()


### Obtaining random numbers and SOAR results from the OceanStateNoise class

Prior to the interpolation work, default values are $q_0 = 10^{-5} \Delta x$ and $L = 0.75 \Delta x$.

In [ ]:
if 'noise' in globals():
    noise.cleanUp()
#reload(OceanStateNoise)

f = 2e-4
staggered = False
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

nx = 210
ny = 210
dx = 100
dy = 100

rand_nx = 30
rand_ny = 30

nx = 60
ny = 60

rand_nx = 20
rand_ny = 20


coarse_dx = dx*nx/rand_nx
coarse_dy = dy*ny/rand_ny
print("(coarse_dx, coarse_dy)", (coarse_dx, coarse_dy))


coarse_data_shape = (rand_ny+4, rand_nx+4)
coarse_eta = np.zeros(coarse_data_shape, dtype=np.float32)
coarse_hu  = np.zeros(coarse_data_shape, dtype=np.float32)
coarse_hv  = np.zeros(coarse_data_shape, dtype=np.float32)
coarse_H   = np.ones((coarse_data_shape[0]+1, coarse_data_shape[1]+1), dtype=np.float32)*10


fine_data_shape = (ny+4, nx+4)
fine_eta = np.zeros(fine_data_shape, dtype=np.float32)
fine_hu  = np.zeros(fine_data_shape, dtype=np.float32)
fine_hv  = np.zeros(fine_data_shape, dtype=np.float32)
fine_H   = np.ones((fine_data_shape[0]+1, fine_data_shape[1]+1), dtype=np.float32)*10

np.random.seed(0)
L0 = 0.75*coarse_dx
noise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, rand_nx, rand_ny, coarse_dx, coarse_dy,
                                        boundaryConditions, staggered, soar_L = L0)

noise.generateNormalDistribution()
normal_dist_numbers = noise.getRandomNumbers()
imshow(normal_dist_numbers, title='N(0,I)')

noise.perturbOceanStateCPU(coarse_eta, coarse_hu, coarse_hv, coarse_H, f,
                           use_existing_GPU_random_numbers=True, 
                           ghost_cells_x=2, ghost_cells_y=2)

imshow3(coarse_eta, coarse_hu, coarse_hv)

coarse_eta[1,:] = coarse_eta[2+rand_ny-1,:]
coarse_eta[-2,:] = coarse_eta[2, :]
coarse_eta[:,1] = coarse_eta[:,2+rand_nx-1]
coarse_eta[:,-2] = coarse_eta[:,2]

# Activate the first ghost cell halo with periodic boundary conditions
coarse_eta[0,:] = coarse_eta[2+rand_ny-2,:]
coarse_eta[-1,:] = coarse_eta[3, :]
coarse_eta[:,0] = coarse_eta[:,2+rand_nx-2]
coarse_eta[:,-1] = coarse_eta[:,3]
imshow(coarse_eta)

In [ ]:
wide_coarse_eta = np.zeros((coarse_eta.shape[0]+2, coarse_eta.shape[1]+2))
wide_coarse_eta[1:-1, 1:-1] = coarse_eta
wide_coarse_eta[0,:] = wide_coarse_eta[2+rand_ny-2,:]
wide_coarse_eta[-1,:] = wide_coarse_eta[5, :]
wide_coarse_eta[:,0] = wide_coarse_eta[:,2+rand_nx-2]
wide_coarse_eta[:,-1] = wide_coarse_eta[:,5]
imshow(wide_coarse_eta)

In [ ]:
def soar(a_x, a_y, b_x, b_y, q0, L):
    dist = np.sqrt( (a_x - b_x)**2 + (a_y - b_y)**2)
    return q0*(1 + dist/L)*np.exp(-dist/L)

In [ ]:
print (coarse_eta.shape)
print(ny, nx)

In [ ]:
for loc_j in range(ny):
    for loc_i in range(nx):
        i = loc_i + 2
        j = loc_j + 2
        
        # Position of cell center in fine grid:
        x = (loc_i + 0.5)*dx
        y = (loc_j + 0.5)*dy
        
        # Location in coarse grid:
        coarse_i = np.floor(x/coarse_dx + 0.5)
        coarse_j = np.floor(y/coarse_dy + 0.5)
        #rint ("(i, x, coarse_i)", (i, x, coarse_i))
        
        coarse_i_index = int(coarse_i + 3)
        coarse_j_index = int(coarse_j + 3)
        
        # Interpolate between the four closest points, with weights according to the SOAR function
        w00 = soar(x, y, (coarse_i+0.5)*coarse_dx, (coarse_j +0.5)*coarse_dy,
                   noise.soar_q0, noise.soar_L)
        w10 = soar(x, y, (coarse_i+1.5)*coarse_dx, (coarse_j +0.5)*coarse_dy,
                   noise.soar_q0, noise.soar_L)
        w01 = soar(x, y, (coarse_i+0.5)*coarse_dx, (coarse_j +1.5)*coarse_dy,
                   noise.soar_q0, noise.soar_L)
        w11 = soar(x, y, (coarse_i+1.5)*coarse_dx, (coarse_j +1.5)*coarse_dy,
                   noise.soar_q0, noise.soar_L)
        sum_weights = w00 + w10 + w01 + w11
        
        sum_weights = 0.0
        contribution = 0.0
        cutoff = 2
        for c_j in range(-cutoff, cutoff+1):
            for c_i in range(-cutoff, cutoff+1):
                weight = soar(x, y, (coarse_i+c_i+0.5)*coarse_dx, (coarse_j+c_j +0.5)*coarse_dy,
                              noise.soar_q0, noise.soar_L)
                contribution += weight*wide_coarse_eta[coarse_j_index+c_j, coarse_i_index+c_i]
                sum_weights += weight
        fine_eta[j,i] = contribution/sum_weights
        
        #fine_eta[j, i] = (w00*coarse_eta[coarse_j_index  , coarse_i_index  ] + \
        #                  w01*coarse_eta[coarse_j_index  , coarse_i_index+1] +
        #                  w10*coarse_eta[coarse_j_index+1, coarse_i_index  ] +
        #                  w11*coarse_eta[coarse_j_index+1, coarse_i_index+1]) / sum_weights

imshow(fine_eta, title='fine_eta')
imshow(coarse_eta[2:-2, 2:-2], title='coarse_eta')

fig = plt.figure(figsize=(6,3))
plt.plot(fine_eta[int(ny/2),:])

This form of interpolation didn't really work... Let's try something else!

In [ ]:
def plotSoar(L=1, q=1):
    fig = plt.figure(figsize=(6,3))
    x = np.linspace(0, 5)
    y = q*(1+x/L)*np.exp(-x/L)
    plt.plot(x,y)
    plt.grid()
plotSoar(L=0.5)
plotSoar(L=0.75)
plotSoar(L=1.0)

print (noise.soar_L)

### New strategy!
* Draw N(0,I) numbers on coarse grid
* Fill sufficient periodic ghost cells
* For each point in fine grid, apply SOAR based on the coarse cut-off!

In [ ]:
cutoff = 2

Qxi = np.zeros((ny+2, nx+2))

# Assume in a GPU setting - we read xi (coarse random numbers) into shared memory with ghostcells
ny_halo = int(rand_ny + (1 + cutoff)*2)
nx_halo = int(rand_nx + (1 + cutoff)*2)
local_xi = np.zeros((ny_halo, nx_halo))
for j in range(ny_halo):
    global_j = j
    global_j = (j - cutoff - 1) % rand_ny
    for i in range(nx_halo):
        global_i = i
        global_i = (i - cutoff - 1) % rand_nx
        
        local_xi[j,i] = normal_dist_numbers[global_j, global_i]

    # Sync threads

    # Loop over fine grid
    for a_y in range(ny+2):
        for a_x in range(nx+2):
            
            # Find index of the coarse cell we are with-in
            x = (a_x - 1 + 0.5)*dx
            y = (a_y - 1 + 0.5)*dy
            
            coarse_i = int(np.floor(x/coarse_dx) + 1 + cutoff)
            coarse_j = int(np.floor(y/coarse_dy) + 1 + cutoff)
            
            start_coarse_i = coarse_i - cutoff
            end_coarse_i   = coarse_i + cutoff + 1
            start_coarse_j = coarse_j - cutoff
            end_coarse_j   = coarse_j + cutoff + 1
            
            #c_y = (coarse_j + 0.5 - 1 - cutoff)*coarse_dy
            #c_x = (coarse_i + 0.5 - 1 - cutoff)*coarse_dx
            #Qxi[a_y, a_x] = np.sqrt((c_x - x)**2 + (c_y - y)**2)
            #Qxi[a_y, a_x] = local_xi[coarse_j, coarse_i]
            #continue
            
            Qx = 0.0
            for c_j in range(start_coarse_j, end_coarse_j):
                for c_i in range(start_coarse_i, end_coarse_i):
                    c_y = (c_j + 0.5 - 1 - cutoff)*coarse_dy
                    c_x = (c_i + 0.5 - 1 - cutoff)*coarse_dx
                    Q = soar(x, y, c_x, c_y, noise.soar_q0, noise.soar_L)
                    Qx += Q*local_xi[c_j, c_i]
            Qxi[a_y, a_x] = Qx
            
            
            # This is a GPU thread (a_x, a_y)
            #local_a_x = a_x + cutoff
            #local_a_y = a_y + cutoff

            #############
            #Qxi[a_y, a_x] = local_xi[local_a_y, local_a_x]
            #continue
            #############

imshow(normal_dist_numbers, title="N(0,I)")
imshow(Qxi, title="Qxi")
fig = plt.figure(figsize=(6,3))
plt.plot(Qxi[int(ny/2),:])
imshow(coarse_eta, title="coarse_eta")

In [ ]:
imshow(normal_dist_numbers, title="N(0,I)")
imshow(Qxi, title="Qxi", figsize=(12,12))
imshow(coarse_eta, title="coarse_eta",  figsize=(12,12))

fig = plt.figure(figsize=(6,3))
plt.plot(Qxi[40,:])


In [ ]:
imshow(normal_dist_numbers, title="N(0,I)")
imshow(Qxi, title="Qxi", figsize=(12,12))
imshow(coarse_eta, title="coarse_eta",  figsize=(12,12))

fig = plt.figure(figsize=(6,3))
plt.plot(Qxi[40,:])


In [ ]:
print (dx, dy)
print (coarse_dx, coarse_dy)
print (noise.soar_L)

## Bi-cubic interpolation
Need one layer of ghost cells for the computations, and one extra ghost cell layer of eta's to obtain geostrophic balance

In [ ]:
print ("hei")

bicubic_matrix = np.matrix([[ 1,  0,  0,  0], 
                            [ 0,  0,  1,  0], 
                            [-3,  3, -2, -1],
                            [ 2, -2,  1,  1]])


print (bicubic_matrix)
print(np.dot(bicubic_matrix, bicubic_matrix.transpose()))
#asff

fine_eta *= 0
linear_eta = fine_eta.copy()


print("fine    (nx, ny): ", (nx, ny))
print("fine    (dx, dy): ", (dx, dy))
print("fine_eta.shape  : ", fine_eta.shape)
print("coarse  (nx, ny): ", (rand_nx, rand_ny))
print("coarse  (dx, dy): ", (coarse_dx, coarse_dy))
print("coarse_eta.shape: ", coarse_eta.shape)

min_rel_x = 10
max_rel_x = -10
min_rel_y = 10
max_rel_y = -10

for loc_j in range(ny+2):

    for loc_i in range(nx+2):
        i = loc_i + 1
        j = loc_j + 1
        
        # Position of cell center in fine grid:
        x = (i - 2 + 0.5)*dx
        y = (j - 2 + 0.5)*dy
        
        # Location in coarse grid (defined in course grid's cell centers)
        coarse_i = int(np.floor(x/coarse_dx + 2 - 0.5))
        coarse_j = int(np.floor(y/coarse_dy + 2 - 0.5))
        coarse_x = (coarse_i - 2 + 0.5)*coarse_dx
        coarse_y = (coarse_j - 2 + 0.5)*coarse_dy
        
        # Defining the coarse grid points on coarse intersections rather than in grid centers.
        #coarse_i = int(np.floor(x/coarse_dx + 1))
        #coarse_j = int(np.floor(y/coarse_dy + 1))
        #coarse_x = (coarse_i - 1)*coarse_dx 
        #coarse_y = (coarse_j - 1)*coarse_dy 

        #print ("(i, x, coarse_i, coarse_x)", (i, x, coarse_i, coarse_x))
        #if loc_i == 0:
        #    print ("--> (j, y, coarse_j, coarse_y)", (j, y, coarse_j, coarse_y))

        
        f00   =  coarse_eta[coarse_j  , coarse_i  ]
        f01   =  coarse_eta[coarse_j+1, coarse_i  ]
        f10   =  coarse_eta[coarse_j  , coarse_i+1]
        f11   =  coarse_eta[coarse_j+1, coarse_i+1]
         
        fx00  = (coarse_eta[coarse_j  , coarse_i+1] - coarse_eta[coarse_j  , coarse_i-1])/2
        fx01  = (coarse_eta[coarse_j+1, coarse_i+1] - coarse_eta[coarse_j+1, coarse_i-1])/2       
        fx10  = (coarse_eta[coarse_j  , coarse_i+2] - coarse_eta[coarse_j  , coarse_i  ])/2    
        fx11  = (coarse_eta[coarse_j+1, coarse_i+2] - coarse_eta[coarse_j+1, coarse_i  ])/2      
        
        fy00  = (coarse_eta[coarse_j+1, coarse_i  ] - coarse_eta[coarse_j-1, coarse_i  ])/2
        fy01  = (coarse_eta[coarse_j+2, coarse_i  ] - coarse_eta[coarse_j  , coarse_i  ])/2       
        fy10  = (coarse_eta[coarse_j+1, coarse_i+1] - coarse_eta[coarse_j-1, coarse_i+1])/2       
        fy11  = (coarse_eta[coarse_j+2, coarse_i+1] - coarse_eta[coarse_j  , coarse_i+1])/2       
        
        fy_10 = (coarse_eta[coarse_j+1, coarse_i-1] - coarse_eta[coarse_j-1, coarse_i-1])/2
        fy_11 = (coarse_eta[coarse_j+2, coarse_i-1] - coarse_eta[coarse_j  , coarse_i-1])/2
        fy20  = (coarse_eta[coarse_j+1, coarse_i+2] - coarse_eta[coarse_j-1, coarse_i+2])/2
        fy21  = (coarse_eta[coarse_j+2, coarse_i+2] - coarse_eta[coarse_j  , coarse_i+2])/2
        
        fxy00 = (fy10 - fy_10)/2
        fxy01 = (fy11 - fy_11)/2
        fxy10 = (fy20 -  fy00)/2
        fxy11 = (fy21 -  fy01)/2
        
        
        f_matrix = np.matrix([[ f00,  f01,  fy00,  fy01],
                              [ f10,  f11,  fy10,  fy11],
                              [fx00, fx01, fxy00, fxy01],
                              [fx10, fx11, fxy10, fxy11] ])
        
        a_matrix = np.dot(bicubic_matrix, np.dot(f_matrix, bicubic_matrix.transpose()))
        
        
        assert coarse_x <= x
        assert coarse_x + coarse_dx >= x
            
        rel_x = (x - coarse_x)/coarse_dx
        rel_y = (y - coarse_y)/coarse_dy
        
        if rel_x < min_rel_x:
            min_rel_x = rel_x
        if rel_x > max_rel_x:
            max_rel_x = rel_x
        if rel_y < min_rel_y:
            min_rel_y = rel_y
        if rel_y > max_rel_y:
            max_rel_y = rel_y
            
        assert rel_x >= 0 and rel_x < 1
        assert rel_y >= 0 and rel_y < 1
        
        x_vec = np.matrix([1.0, rel_x, rel_x*rel_x, rel_x*rel_x*rel_x])
        y_vec = np.matrix([1.0, rel_y, rel_y*rel_y, rel_y*rel_y*rel_y]).transpose()
        

        interpolation_mode = 2
        
        if interpolation_mode == 0:
            # Flat average:
            fine_eta[j,i] = 0.25*(f00 + f01 + f10 + f11)
        
        elif interpolation_mode == 1:
            # Linear interpolation:
            #fine_eta[j,i] = f00 + rel_x*(f10 - f00) + rel_y*(f01 - f00)
            fine_eta[j,i] = f00*(1-rel_x)*(1-rel_y) + f10*rel_x*(1-rel_y) + f01*(1-rel_x)*rel_y + f11*rel_x*rel_y
            
        
        elif interpolation_mode == 2:
            fine_eta[j,i] = np.dot(x_vec, np.dot(a_matrix, y_vec))
            
        linear_eta[j,i] = f00*(1-rel_x)*(1-rel_y) + f10*rel_x*(1-rel_y) + f01*(1-rel_x)*rel_y + f11*rel_x*rel_y

        
        #fine_eta[j, i] = (w00*coarse_eta[coarse_j_index  , coarse_i_index  ] + \
        #                  w01*coarse_eta[coarse_j_index  , coarse_i_index+1] +
        #                  w10*coarse_eta[coarse_j_index+1, coarse_i_index  ] +
        #                  w11*coarse_eta[coarse_j_index+1, coarse_i_index+1]) / sum_weights

imshow(fine_eta, title="fine_eta")
imshow(fine_eta[2:-2, 2:-2], title="fine_eta[2:-2, 2:-2]", figsize=(12,12))

fig = plt.figure(figsize=(9,4))
plt.plot(linear_eta[int(nx/2),:])
plt.plot(fine_eta[int(ny/2),:])
plt.plot(fine_eta[int(ny/1),:])
plt.plot(fine_eta[int(ny/4),:])
plt.grid()

imshow(coarse_eta[2:-2, 2:-2], title="coarse_eta", figsize=(12,12))

print("(min_rel_x, max_rel_x)", (min_rel_x, max_rel_x))
print("(min_rel_y, max_rel_y)", (min_rel_y, max_rel_y))

In [ ]:
inner_cpu = fine_eta[2:-2,2:-2]
imshow(inner_cpu[1::3, 1::3] - coarse_eta[2:-2, 2:-2])

In [ ]:
imshow(coarse_eta[2:-2, 2:-2], title="coarse_eta", figsize=(12,12), interpolation="bicubic")


In [ ]:
x_vec = np.matrix([1, rel_x, rel_x*rel_x, rel_x*rel_x*rel_x])
y_vec = np.matrix([1, rel_y, rel_y*rel_y, rel_y*rel_y*rel_y]).transpose()
print (x_vec)
print(y_vec)

In [ ]:
a = np.zeros((10,10))
for i in range(10):
    for j in range(10):
        a[j,i] = j*10 + i
        
print(a[0::2, 0::2])

In [ ]:
print (nx, ny)
print (rand_nx, rand_ny)


interior_fine_eta = fine_eta[2:-2, 2:-2]
interior_course_eta = coarse_eta[2:-2, 2:-2]

print(interior_fine_eta.shape)
print(interior_course_eta.shape)
print(interior_fine_eta[3::7, 3::7].shape)

print(np.max(np.abs(interior_course_eta - interior_fine_eta[3::7, 3::7])))

In [ ]:
np.ceil(4.234)

# Testing the new class


In [ ]:
if 'newnoise' in globals():
    newnoise.cleanUp()
reload(OceanStateNoise)
nx, ny = 80, 80
dx, dy = 100.0, 100.0
interpolation_factor = 5

newnoise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, \
                                           nx, ny, dx, dy, \
                                           Common.BoundaryConditions(2,2,2,2),
                                           False,
                                           interpolation_factor=interpolation_factor)

dataShape = (ny+4, nx+4)
buf = np.zeros(dataShape, dtype=np.float32)
H_buffer = np.ones((ny+5, nx+5), dtype=np.float32) * 10

sim = CDKLM16.CDKLM16(gpu_ctx, buf.copy(),buf.copy(),buf.copy(), H_buffer, \
                      nx, ny, dx, dy, 0.5, 9.81, 0.002, 0  )

newnoise.perturbSim(sim)

coarse_vals = newnoise.getCoarseBuffer()
#imshow(coarse_vals, title="Coarse GPU")

#imshow(coarse_vals[2:-2, 2:-2], title="Coarse GPU (interior only)")

eta, hu, hv = sim.download(interior_domain_only=False)
imshow3(eta, hu, hv, title="Ocean state GPU")

e = np.zeros(dataShape, dtype=np.float32)
u = np.zeros(dataShape, dtype=np.float32)
v = np.zeros(dataShape, dtype=np.float32)
#v = np.zeros((28, 28), dtype=np.float32)
newnoise.perturbOceanStateCPU(e,u,v,H_buffer, sim.f, \
                              use_existing_GPU_random_numbers=True,
                             ghost_cells_x=2, ghost_cells_y=2)
#newnoise.perturbEtaCPU(e, True, ghost_cells_x=2, ghost_cells_y=2)
imshow3(e,u,v, title="Ocean state CPU")

imshow3(coarse_vals[2:-2, 2:-2], eta[2:-2, 2:-2], e[2:-2, 2:-2], title="eta {coarse, GPU, CPU}")

imshow(coarse_vals[2:-2, 2:-2], title="coarse_eta[2:-2, 2:-2]", figsize=(12,12))
imshow(e[2:-2, 2:-2], title="fine_eta[2:-2, 2:-2]", figsize=(12,12))
imshow(u[2:-2, 2:-2], title="fine_hu[2:-2, 2:-2]", figsize=(12,12))




In [ ]:
inner_gpu = eta[2:-2,2:-2]
imshow3(inner_gpu[2::5, 2::5],  coarse_vals[2:-2, 2:-2],
        inner_gpu[2::5, 2::5] - coarse_vals[2:-2, 2:-2])

In [ ]:
print(coarse_vals.shape)
print(eta.shape)
imshow(coarse_vals[2:-2,2:-2] - eta[2:-2,2:-2])
imshow(e[2:-2,2:-2] - eta[2:-2,2:-2])
imshow3(e - eta, u - hu, v - hv)


# Applying a covariance structure to a variable perturbation

We want to perturb $\eta$ with a model error $\Delta \eta$, so that the covariance of $\Delta \eta$ becomes $Q$. 

Let $\xi$ be a vector of size $n_x n_y$ (same as the vector $\eta$), with $\xi_i \sim N(0,1)$. The perturbation can be generated as $\Delta \eta = Q^{1/2} \xi$, where 
$$ Q^{1/2}(a,b) = q_0 \left[ 1 + \frac{dist(a,b)}{L} \right] \exp \left\{ - \frac{dist(a,b)}{L} \right\},$$
in which $a$ and $b$ are cells, $L$ is a length scale, and $q_0$ is a scale on the amplitude.

**Recall**, if $Z$ is a random variable from $N(0,1)$ (mean 0 and variance 1), then $ X = \sigma Z + \mu$ is a random variable with mean $mu$ and variance $\sigma^2$.

In [ ]:

def SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L):
    dist = np.sqrt( dx*dx*(a_x - b_x)**2  +  dy*dy*(a_y - b_y)**2)
    return q0*(1.0 + dist/L)*np.exp(-dist/L)

def createQMatrix(nx, ny, dx=1, dy=1, q0=1, L=1):
    Q = np.zeros((ny*nx, ny*nx))
    for a_y in range(ny):
        for a_x in range(nx):
            j = a_y*nx + a_x
            for b_y in range(ny):
                for b_x in range(nx):
                    i = b_y*nx + b_x
                    Q[j, i] = SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L)
    return Q

def applyQ(xi, dx=1, dy=1, q0=0.1, L=1):
    Qxi = np.zeros_like(xi)
    (ny, nx) = xi.shape
    for a_y in range(ny):
        for a_x in range(nx):
            # This is a OpenCL thread
            Qx = 0
            for b_y in range(ny):
                for b_x in range(nx):
                    Q = SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L)
                    Qx += Q*xi[b_y, b_x]
            Qxi[a_y, a_x] = Qx
    return Qxi


def applyQfast(xi, dx=1, dy=1, q0=0.1, L=1, cutoff=10):
    Qxi = np.zeros_like(xi)
    (ny, nx) = xi.shape
    for a_y in range(ny):
        for a_x in range(nx):
            # This is a OpenCL thread (a_x, a_y)
            Qx = 0
            start_b_y = max(0, a_y - cutoff)
            end_b_y =  min(ny, a_y + cutoff+1)
            start_b_x = max(0, a_x - cutoff)
            end_b_x =  min(nx, a_x + cutoff+1)
            
            for b_y in range(start_b_y, end_b_y):
                for b_x in range(start_b_x, end_b_x):
                    Q = SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L)
                    Qx += Q*xi[b_y, b_x]
            Qxi[a_y, a_x] = Qx
    return Qxi

def applyQfastPeriodic(xi, dx=1, dy=1, q0=0.1, L=1, cutoff=5):
    # Assume in a GPU setting - we read xi into shared memory with ghostcells
    (ny, nx) = xi.shape
    ny_halo = int(ny + cutoff*2)
    nx_halo = int(nx + cutoff*2)
    local_xi = np.zeros((ny_halo, nx_halo))
    for j in range(ny_halo):
        global_j = (j - cutoff) % ny
        for i in range(nx_halo):
            global_i = (i - cutoff) % nx
            local_xi[j,i] = xi[global_j, global_i]
    # Sync threads
    
    Qxi = np.zeros_like(xi)
    for a_y in range(ny):
        for a_x in range(nx):
            # This is a OpenCL thread (a_x, a_y)
            local_a_x = a_x + cutoff
            local_a_y = a_y + cutoff
            
            start_b_y = local_a_y - cutoff
            end_b_y =  local_a_y + cutoff+1
            start_b_x = local_a_x - cutoff
            end_b_x =  local_a_x + cutoff+1
            
            Qx = 0
            for b_y in range(start_b_y, end_b_y):
                for b_x in range(start_b_x, end_b_x):
                    Q = SOAR_Q(local_a_x, local_a_y, b_x, b_y, dx, dy, q0, L)
                    Qx += Q*local_xi[b_y, b_x]
            Qxi[a_y, a_x] = Qx
    return Qxi

    
nx = 40
ny = 40
L = 0.75
q0 = 0.05
fullQ = False
if fullQ:
    Q = createQMatrix(nx, ny, L=L, q0=q0)
    fig = plt.figure(figsize=(4,4))
    plt.imshow(Q, interpolation="None")
    plt.colorbar()
    fig = plt.figure(figsize=(4,4))
    plt.imshow(np.dot(Q,Q), interpolation="None")
    plt.colorbar()
    print "norm of Q - Q.T: ", np.linalg.norm(Q - Q.T)
    print "Q[5,5]: ", Q[5,5]
    print "max(Q): ", np.max(Q)

#xi = np.random.rand(ny, nx)
xi = np.random.normal(size=(ny, nx))
fig = plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.imshow(xi, interpolation="None")
plt.title('Xi')
plt.colorbar()
Qxi = applyQ(xi, L=L, q0=q0)
plt.subplot(1,3,2)
plt.imshow(Qxi, interpolation="None")
plt.title('Full Q*xi')
plt.colorbar()
Qxifast = applyQfast(xi,  L=L, q0=q0, cutoff=3)
plt.subplot(1,3,3)
plt.imshow(Qxifast, interpolation="None")
plt.title('Q*xi with cutoff = 3')
plt.colorbar()

# Investigate the effect of cutoff
print "norm of Qxi: ", np.linalg.norm(Qxi)
for cutoff in range(min(nx, 8)):
    Qxifast = applyQfast(xi,  L=L, q0=q0, cutoff=cutoff)
    print "Diff with cutoff = " + str(cutoff) + ": ", np.linalg.norm(Qxi - Qxifast)
    print "\tMax diff:     ", np.max(np.abs(Qxi - Qxifast))
    print "\tMax rel diff: ", np.max(np.abs((Qxi - Qxifast)))*100/np.linalg.norm(Qxi)
    
cutoff = 5
QxiPeriodic = applyQfastPeriodic(xi,  L=L, q0=q0, cutoff=cutoff)
fig = plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(QxiPeriodic, interpolation="None")
plt.title('Periodic Q*xi, cutoff = ' + str(cutoff))
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(QxiPeriodic-Qxi, interpolation="None", vmin=np.min(Qxifast), vmax=np.max(Qxifast))
plt.title('Periodic Q*xi - full Q*xi')
plt.colorbar()

fig = plt.figure(figsize=(8,2))
plt.subplot(1,2,1)
plt.hist(xi.flatten(), bins=50, normed=True)
plt.title("Distribution of xi values")
plt.subplot(1,2,2)
plt.hist(Qxi.flatten(), bins=50, normed=True)
plt.title("Distribution of full Qxi values")


In [ ]:
fig = plt.figure(figsize=(12,6))

for cutoff in range(2,5):
    # Visualize a given cutoff value:
    

    Qxifast = applyQfast(xi,  L=L, q0=q0, cutoff=cutoff)
    plotNo = cutoff - 2
    
    plt.subplot(2,3,plotNo + 1)
    plt.imshow(Qxifast, interpolation="None")
    plt.title('Cutoff ' + str(cutoff))
    plt.colorbar()
    
    plt.subplot(2,3,plotNo + 4)
    plt.imshow(Qxifast-Qxi, interpolation="None", vmin=np.min(Qxifast), vmax=np.max(Qxifast))
    plt.title('Cutoff diff from full')
    plt.colorbar()
    

plt.suptitle('Different cutoff sizes')


In [ ]:
#def applyQfastPeriodic(xi, dx=1, dy=1, q0=0.1, L=1, cutoff=5):

def obtainOceanPerturbations(nx, ny, seed, f, H=10, beta=0, g=9.81,
                             ghosts_x=2, ghosts_y=2, dx=1, dy=1, 
                             q0=0.1, L=1.0, cutoff=5):
    xi = np.zeros((ny, nx))
    updateRandom(xi, seed)
    
    ####
    # Global sync
    ####
    
    d_eta = applyQfastPeriodic(xi, dx=dx, dy=dy, q0=q0, L=L, cutoff=cutoff)
    
    ####
    # Global sync (currently)
    #     Can be made into a local sync, as long as d_eta is given 
    #     periodic overlap (1 more global computated ghost cell)
    ####
    
    
    d_hu = np.zeros((ny, nx))
    d_hv = np.zeros((ny, nx))
    
    ### Find H_mid:
    # Read global H (def on intersections) to local, find H_mid
    # The local memory can then be reused to something else (perhaps use local_d_eta before computing local_d_eta?)
    #
    # Here, we just set it to 10
    H_mid = H
    ####
    # Local sync
    ####
    
    local_d_eta = np.zeros((ny+2, nx+2))
    for j in range(ny + 2):
        global_j = (j - 1) % ny
        for i in range(nx + 2):
            global_i = (i - 1) % nx
            local_d_eta[j,i] = d_eta[global_j, global_i]
    
    ####
    # LOCAL sync
    ####
    
    

    for j in range(0, ny):
        local_j = j + 1
        coriolis = f + beta*local_j*dy
        for i in range(0, nx):
            local_i = i + 1
            h_mid = local_d_eta[local_j,local_i] + H_mid
            
            eta_diff_y = (local_d_eta[local_j+1, local_i] - local_d_eta[local_j-1, local_i])/(2.0*dy)
            d_hu[j,i] = -(g/coriolis)*h_mid*eta_diff_y
            
            eta_diff_x = (local_d_eta[local_j, local_i+1] - local_d_eta[local_j, local_i-1])/(2.0*dx)
            d_hv[j,i] = (g/coriolis)*h_mid*eta_diff_x   
    
    return d_eta, d_hu, d_hv
    


nx = 50
ny = 50
ghosts_x = 0
ghosts_y = 0
halo_nx = nx + ghosts_x*2
halo_ny = ny + ghosts_y*2
dx = 100.0
dy = 100.0
H = 60

f = 0.02
seeds = np.random.rand(ny, nx/2)*0x7fffffff

q0 = dx/1000000
L = 0.75*dx
cutoff = 2

d_eta, d_hu, d_hv = obtainOceanPerturbations(nx, ny, seeds, f, H=H, ghosts_x=ghosts_x, ghosts_y=ghosts_y, dx=dx, dy=dy,
                                             q0=q0, L=L, cutoff=cutoff)

fig = plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.imshow(d_eta, interpolation="None", origin="lower")
plt.title('$\Delta \eta$')
plt.colorbar()

plt.subplot(1,3,2)
plt.imshow(d_hu, interpolation="None", origin="lower")
plt.title('$\Delta hu$')
plt.colorbar()

plt.subplot(1,3,3)
plt.imshow(d_hv, interpolation="None", origin="lower")
plt.title('$\Delta hv$')
plt.colorbar()

fig = plt.figure(figsize=(5,5))
X,Y = np.meshgrid(np.arange(0, nx, 1.0), np.arange(0, ny, 1.0))
plt.quiver(X, Y, d_hu, d_hv)



# GPU implementation

Some considerations have to be done with respect to boundary conditions.

In order to generate perturbed eta with ***double periodic boundary conditions and assuming non-staggered grids***, we would 
- generate $\xi$ as $(nx, ny)$ random normal distributed numbers
- apply periodic boundary conditions to $\xi$ (in local memory), so that it gets the size 
$$(nx+2(1+cutoff), ny + 2(1+cutoff))$$
- apply SOAR to $\xi$, giving d_eta on $(nx + 2, ny + 2)$, where the outer cells fulfills the periodicity.
- find d_hu and d_hv through the geostrophic balance, sizes $(nx, ny)$, and use only the inner $(nx, ny)$ of d_eta.

Now, what happens if we have ***staggered grids***? In order to calculate d_hu and d_hv, we require the same number of cells as in the non-staggered case, so the only difference is the computation of geostrophic balance. The total required data memory layout is the same, and d_eta does not need to be modified.

How about ***other boundary conditions***? 
First, how to give a consistent covariance structure of d_eta? By using extra random numbers. Internally, the computational domain of wall and open boundary conditions are the same (here $(nx, ny)$). The tactic would be to produce $\xi$ $$(nx+2(1+cutoff), ny + 2(1+cutoff))$$ random numbers at once, where no boundary conditions are applied to $\xi$. Applying SOAR would give d_eta of the same size as above, and final d_eta, d_hu and d_hv of sizes $(nx, ny)$.

***Mixed boundary conditions***, where they are periodic in only one direction. The only difference will 

***Common for all boundary conditions*** would be the final step, namely 
$$\eta += \Delta \eta, \quad hu += \Delta hu, \quad hv += \Delta hv,$$
where each perturbation is only added on the computational domain. 
Boundary conditions would have to be applied to all three fields after the perturbation is added in any case.

In [ ]:
if 'noise' in globals():
    noise.cleanUp()
reload(OceanStateNoise)
def compareDeltaEta(noise):
    etaCPU = np.zeros((noise.ny, noise.nx))
    HCPU = np.ones((noise.ny+1, noise.nx+1))*10.0
    etaGPU = Common.CUDAArray2D(gpu_stream, noise.nx, noise.ny, 0, 0, etaCPU)
    huGPU = Common.CUDAArray2D(gpu_stream, noise.nx, noise.ny, 0, 0, etaCPU)
    hvGPU = Common.CUDAArray2D(gpu_stream, noise.nx, noise.ny, 0, 0, etaCPU)
    HGPU = Common.CUDAArray2D(gpu_stream, noise.nx+1, noise.ny+1, 0, 0, HCPU)
    f, beta, g = 0.02, 0.0, 9.81
    
    #noise.generateNormalDistribtion()
    noise.perturbOceanState(etaGPU, huGPU, hvGPU, HGPU, f, beta, g)
    noise.perturbEtaCPU(etaCPU, use_existing_GPU_random_numbers=True)
    print "perturbed etaCPU - min/max(etaCPU): ", (np.min(etaCPU), np.max(etaCPU))
    etaFromGPU = etaGPU.download(gpu_stream)
    etaGPU.release()
    huGPU.release()
    hvGPU.release()
    
    eta_max = max(np.max(np.abs(etaCPU)), np.max(np.abs(etaFromGPU)))
    
    fig = plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    im = plt.imshow(etaCPU, interpolation="None", origin="lower")
    plt.title('$\Delta \eta$ from CPU')
    im.set_clim(-eta_max, eta_max)
    plt.colorbar()

    plt.subplot(1,3,2)
    im = plt.imshow(etaFromGPU, interpolation="None", origin="lower")
    plt.title('$\Delta \eta$ from GPU')
    im.set_clim(-eta_max, eta_max)
    plt.colorbar()

    plt.subplot(1,3,3)
    plt.imshow(etaCPU-etaFromGPU, interpolation="None", origin="lower")
    plt.title('Diff between CPU and GPU')
    plt.colorbar()
    
    if noise.periodicNorthSouth and noise.periodicEastWest:
        plt.suptitle("Full periodic")
    elif noise.periodicNorthSouth:
        plt.suptitle("Periodic north-south")
    elif noise.periodicEastWest:
        plt.suptitle("Periodic east-west")
    else:
        plt.suptitle("Non-periodic")
    return etaFromGPU
    
nx, ny = 50, 50
dx, dy = 1.0, 1.0
staggered = False
bcs = [Common.BoundaryConditions(2,2,2,2), Common.BoundaryConditions(1,1,1,1),
      Common.BoundaryConditions(1,2,1,2), Common.BoundaryConditions(2,1,2,1)]
#bcs = [Common.BoundaryConditions(2,2,2,2)]
eta = None
for bc in bcs:
    noise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, nx, ny, dx, dy, bc, staggered)  
    eta = compareDeltaEta(noise)
fig = plt.figure(figsize=(4,2))
plt.hist(eta.flatten(), bins=50, normed=True)
print "ok?"
print eta[0,0:20]
mod_test = np.zeros(20)
for i in range(20):
    tx = i % 16
    mod_test[i] = (tx-3)%10
print mod_test
print eta[-1,-1]
print eta.shape


In [ ]:
if 'noise' in globals():
    noise.cleanUp()
reload(OceanStateNoise)
def compareOceanNoise(noise):
    etaCPU = np.zeros((noise.ny, noise.nx))
    huCPU = np.zeros((noise.ny, noise.nx))
    hvCPU = np.zeros((noise.ny, noise.nx))
    HCPU = np.ones((noise.ny+1, noise.nx+1))*10.0

    etaGPU = Common.CUDAArray2D(gpu_stream, noise.nx, noise.ny, 0, 0, etaCPU)
    huGPU = Common.CUDAArray2D(gpu_stream, noise.nx, noise.ny, 0, 0, etaCPU)
    hvGPU = Common.CUDAArray2D(gpu_stream, noise.nx, noise.ny, 0, 0, etaCPU)
    HGPU = Common.CUDAArray2D(gpu_stream, noise.nx+1, noise.ny+1, 0, 0, HCPU)
    f, beta, g = 0.02, 0.0, 9.81
    
    #noise.generateNormalDistribtion()
    noise.perturbOceanState(etaGPU, huGPU, hvGPU, HGPU, f, beta, g)
    noise.perturbOceanStateCPU(etaCPU, huCPU, hvCPU, HCPU, f, beta, g, use_existing_GPU_random_numbers=True)
    #print "perturbed etaCPU - min/max(etaCPU): ", (np.min(etaCPU), np.max(etaCPU))
    eta = etaGPU.download(gpu_stream)
    hu = huGPU.download(gpu_stream)
    hv = hvGPU.download(gpu_stream)
    etaGPU.release()
    huGPU.release()
    hvGPU.release()
    
    eta_max = max(np.max(np.abs(etaCPU)), np.max(np.abs(eta)))
    hu_max = max(np.max(np.abs(huCPU)), np.max(np.abs(hu)))
    hv_max = max(np.max(np.abs(hvCPU)), np.max(np.abs(hv)))
    
    
    # ----- CPU ----------
    fig = plt.figure(figsize=(12,12))
    plt.subplot(3,3,1)
    im = plt.imshow(etaCPU, interpolation="None", origin="lower")
    plt.title('$\Delta \eta$ from CPU')
    im.set_clim(-eta_max, eta_max)
    plt.colorbar()
    
    plt.subplot(3,3,2)
    im = plt.imshow(huCPU, interpolation="None", origin="lower")
    plt.title('$\Delta hu$ from CPU')
    im.set_clim(-hu_max, hu_max)
    plt.colorbar()
    
    plt.subplot(3,3,3)
    im = plt.imshow(hvCPU, interpolation="None", origin="lower")
    plt.title('$\Delta hv$ from CPU')
    im.set_clim(-hv_max, hv_max)
    plt.colorbar()

    ## ---------- GPU ------    
    plt.subplot(3,3,4)
    im = plt.imshow(eta, interpolation="None", origin="lower")
    plt.title('$\Delta \eta$ from GPU')
    im.set_clim(-eta_max, eta_max)
    plt.colorbar()
    
    plt.subplot(3,3,5)
    im = plt.imshow(hu, interpolation="None", origin="lower")
    plt.title('$\Delta hu$ from GPU')
    im.set_clim(-hu_max, hu_max)
    plt.colorbar()
    
    plt.subplot(3,3,6)
    im = plt.imshow(hv, interpolation="None", origin="lower")
    plt.title('$\Delta hv$ from GPU')
    im.set_clim(-hv_max, hv_max)
    plt.colorbar()

    # ------ Diff ----------
    
    plt.subplot(3,3,7)
    plt.imshow(etaCPU-eta, interpolation="None", origin="lower")
    plt.title('Diff $\eta$ between CPU and GPU')
    plt.colorbar()
    
    plt.subplot(3,3,8)
    plt.imshow(huCPU-hu, interpolation="None", origin="lower")
    plt.title('Diff $hu$ between CPU and GPU')
    plt.colorbar()
    
    plt.subplot(3,3,9)
    plt.imshow(hvCPU-hv, interpolation="None", origin="lower")
    plt.title('Diff $hv$ between CPU and GPU')
    plt.colorbar()
    
    supertitle="Non-periodic"
    if noise.periodicNorthSouth and noise.periodicEastWest:
        supertitle = "Full periodic"
    elif noise.periodicNorthSouth:
        supertitle = "Periodic north-south"
    elif noise.periodicEastWest:
        supertitle = "Periodic east-west"
    plt.suptitle(supertitle)

    fig = plt.figure(figsize=(4,4))
    X,Y = np.meshgrid(np.arange(0, noise.nx, 1.0), np.arange(0, noise.ny, 1.0))
    plt.quiver(X, Y, hu, hv)
    plt.suptitle(supertitle)
    return eta
    
    
nx, ny = 50, 50
dx, dy = 1.0, 1.0
staggered = False
bcs = [Common.BoundaryConditions(2,2,2,2), Common.BoundaryConditions(1,1,1,1),
      Common.BoundaryConditions(1,2,1,2), Common.BoundaryConditions(2,1,2,1)]
#bcs = [Common.BoundaryConditions(2,2,2,2)]
eta = None
for bc in bcs:
    noise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, nx, ny, dx, dy, bc, staggered)  
    eta = compareOceanNoise(noise)
fig = plt.figure(figsize=(4,2))
plt.hist(eta.flatten(), bins=50, normed=True)
print "ok?"


## Simulator with model error

First, let's try to start a model with a field that is perturbed with relatively large $q_0$ (noise with large amplitude).

Drifters are added as a way to visualize the velocity field, but in order to see anything exciting, their sensitivity should be increase (`drifters.setSensitivity(50)` or similar).
Keep in mind that if $hu = 1$ and $H = 10$m, we get $u = 0.1 m/s$ which means it will stay within its $100m \times 100m$ grid cell for (potentially) 1000 seconds :)

In [ ]:
def plotOceanState(sim, ghosts, supertitle):
    eta, hu, hv = sim.download()
    eta = eta[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]]
    hu = hu[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]]
    hv = hv[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]]
    
    fig = plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    im = plt.imshow(eta, interpolation="None", origin="lower")
    plt.title('$\eta$')
    #im.set_clim(-eta_max, eta_max)
    plt.colorbar()
    
    plt.subplot(1,3,2)
    im = plt.imshow(hu, interpolation="None", origin="lower")
    plt.title('$hu$')
    #im.set_clim(-hu_max, hu_max)
    plt.colorbar()
    
    plt.subplot(1,3,3)
    im = plt.imshow(hv, interpolation="None", origin="lower")
    plt.title('$hv$')
    #im.set_clim(-hv_max, hv_max)
    plt.colorbar()
    plt.suptitle(supertitle)
    
    fig = plt.figure(figsize=(3,3))
    X,Y = np.meshgrid(np.arange(0, sim.nx, 1.0), np.arange(0, sim.ny, 1.0))
    plt.quiver(X, Y, hu, hv)
    plt.suptitle(supertitle)
    
    max_u = max(np.max(hu/(eta+10) + np.sqrt(sim.g*(eta+10))),
                np.max(hu/(eta+10) - np.sqrt(sim.g*(eta+10)))  )
    max_v = max(np.max(hv/(eta+10) + np.sqrt(sim.g*(eta+10))),
                np.max(hv/(eta+10) - np.sqrt(sim.g*(eta+10)))  )
    dt_max = 0.25*min(sim.dx/max_u, sim.dy/max_v)
    print "Finding maximum possible dt: ", dt_max


nx, ny = 40, 40 # 100, 100
dx, dy = 100.0 ,100.0
dt = 0.5

f, beta = 0.02, 0.0
g = 9.81
waterdepth = 10.0
r = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref, y_zero_ref = 2, 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

# Input data
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');
Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterdepth


# Required for anim/plotting:
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

staggered = False

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
if 'noise' in globals():
    noise.cleanUp()
    
reload(CDKLM16)
reload(OceanStateNoise)
# Creating sim
sim = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False)

# Adding drifters:
numParticles = 100
gpuParticles = GPUDrifterCollection.GPUDrifterCollection(gpu_ctx, numParticles,
                                                         observation_variance=0.0,
                                                         boundaryConditions=sim.boundary_conditions,
                                                         domain_size_x=sim.nx*sim.dx,
                                                         domain_size_y=sim.ny*sim.dy)
gpuParticles.initializeUniform()
gpuParticles.setSensitivity(10)
sim.attachDrifters(gpuParticles)

q0 = 100*dx/100000
noise = OceanStateNoise.OceanStateNoise.fromsim(sim, soar_q0=q0)
noise.perturbSim(sim)
plotOceanState(sim, ghosts, "Initial conditions with noise")


### Animating simulation from the above state

fig = plt.figure(figsize=(8, 10))
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                                u0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                                v0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
plotter.showDrifters(sim.drifters)
T = 100
sub_t = 200*dt
def animate(i):
    if (i>0):
        t = sim.step(sub_t)
    else:
        t = 0.0

    # Downscale the fields since they are quite intense :P    
    scale = 0.1
    eta1, hu1, hv1 = sim.download()
    plotter.plot(scale*eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                 scale*hu1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                 scale*hv1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]]);
    plotter.showDrifters(sim.drifters)
         
    fig.suptitle("CDKLM16 from a chaotic state = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
                     
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim


## Test simulation with model errors
Starting from lake-at-rest and adding some model errors for every time step. Fun fun fun!

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
if 'noise' in globals():
    noise.cleanUp()

reload(Common)
reload(CDKLM16)
reload(OceanStateNoise)

q0 = 0.1*dx/100000
#noise = OceanStateNoise.OceanStateNoise.fromsim(sim, soar_q0=q0)

sim = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)

# Adding drifters:
numParticles = 100
gpuParticles = GPUDrifterCollection.GPUDrifterCollection(gpu_ctx, numParticles,
                                                         observation_variance=0.0,
                                                         boundaryConditions=sim.boundary_conditions,
                                                         domain_size_x=sim.nx*sim.dx,
                                                         domain_size_y=sim.ny*sim.dy)
gpuParticles.initializeUniform()
#gpuParticles.setSensitivity(10)
sim.attachDrifters(gpuParticles)


### Animating simulation from the above state

fig = plt.figure(figsize=(8, 10))
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                                u0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                                v0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
plotter.showDrifters(sim.drifters)
T = 200
sub_t = 50*dt
def animate(i):
    if (i>0):
        t = sim.step(sub_t)
        
        # Add this line to generate new seed from the CPU every superstep
        #sim.small_scale_model_error.resetSeed()
        
        # Add this block to generate new seed from the CPU every timestep
        #for j in range(50):
        #    t = sim.step(dt)
        #    sim.small_scale_model_error.resetSeed()

    else:
        t = 0.0

    scale = 1.0
    eta1, hu1, hv1 = sim.download()
    plotter.plot(scale*eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                 scale*hu1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], 
                 scale*hv1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]]);
    plotter.showDrifters(sim.drifters)
         
    fig.suptitle("CDKLM16 from rest with model error = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    if (i%10 == 0):
        meanEta = np.mean(eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
        minEta = np.min(eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
        maxEta = np.max(eta1[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]])
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMean eta: " + str([meanEta, minEta, maxEta]) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
                     
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim


## Investigating $\Delta \eta$ mean value

The trend in the perturbation is that the $\eta$-field is perturbed with negative mean. This will lead to a significant loss of mass across several timesteps.

In [ ]:
def evalMean(sim):
    eta, hu, hv = sim.download()
    eta = eta[2:-2, 2:-2]
    meanEta = np.mean(eta)
    return meanEta

if 'sim1' in globals():
    sim1.cleanUp()
if 'noise1' in globals():
    noise1.cleanUp()
if 'sim2' in globals():
    sim2.cleanUp()
if 'noise2' in globals():
    noise2.cleanUp()
reload(CDKLM16)
reload(OceanStateNoise)
# Creating sim
sim1 = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False)
sim2 = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False)



q0 = 100*dx/100000
noise1 = OceanStateNoise.OceanStateNoise.fromsim(sim1, soar_q0=q0)
noise2 = OceanStateNoise.OceanStateNoise.fromsim(sim2, soar_q0=q0)
meanT = 5000
means1 = np.zeros(meanT)
means2 = np.zeros(meanT)
for i in range(meanT):
    for j in range(50):
        noise1.perturbSim(sim1)
        noise2.perturbSim(sim2)
        noise2.resetSeed()
    means1[i] = evalMean(sim1)
    means2[i] = evalMean(sim2)
fig = plt.figure(figsize=(6,3))
plt.plot(means1, label="no reset")
plt.plot(means2, label="reset")
plt.legend(loc=0)
plt.grid()
print (sim1.nx, sim1.ny)

In [ ]:
# Looking at autocorrelation within a cell:

def autocorr(x, k, mean=None, var=None):
    R = np.zeros(k)
    n = len(x)
    if mean is None:
        mean = np.mean(x)
    if var is None:
        var = np.var(x)
    for i in range(k):
        r = 0.0
        for t in range(n-i):
            r += (x[t] - mean)*(x[t+i] - mean)
        R[i] = r/((n-i)*var)
    return R

if 'sim2' in globals():
    sim2.cleanUp()
if 'noise2' in globals():
    noise2.cleanUp()
reload(CDKLM16)
reload(OceanStateNoise)
reload(Common)
# Creating sim
sim2 = CDKLM16.CDKLM16(gpu_ctx, eta0, u0, v0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions,
                      write_netcdf=False)



q0 = 100*dx/100000
noise2 = OceanStateNoise.OceanStateNoise.fromsim(sim2, soar_q0=q0)
noise2.resetSeed()
autocorrN = 10000
for rounds in range(5):
    for i in range(100000):
        noise2.generateNormalDistribution()
    print "round " + str(rounds)
    u = np.zeros((autocorrN, 9))
    for i in range(autocorrN):
        noise2.generateNormalDistribution()
        generatedU = noise2.random_numbers.download(noise2.gpu_stream)
        for j in range(9):
            u[i,j] = generatedU[j,j]


    fig = plt.figure(figsize=(4,2))
    for j in range(9):
        lag = autocorr(u[:,j], 100)
        plt.plot(np.abs(lag))
        print "mean row " + str(j), np.mean(u[:,j])
    print "total mean: ", np.mean(u)
    print "min/max: ", np.min(u), np.max(u)
    plt.title("autocorr")
    plt.grid()
    
    fig = plt.figure(figsize=(4,2))
    cov = np.cov(u, rowvar=False)
    print cov.shape
    plt.imshow(cov)
#np.u


In [ ]:
print 1103515245
print 2**31*0.51386433
print 2**31
print np.float64(2147483648.0)

print np.int16(1103515245)
print (0x7fffffff)

a = np.double(3.14)
print a
print isinstance(a, np.float64)
type(a)